## 手机游戏道具推荐与一般商品推荐的不同

1. 玩家行为数据中包含大量事件上下文。普通互联网产品的事件一般仅包括用户ID、用户行为ID和时间戳等内容。网络游戏的事件不仅包含上述内容,还包括 事件发生环境、前置事件、后续事件等事件上下文信息。
2. 玩家行为数据中包含丰富的隐性事件。玩家在游戏虚拟世界中会留下大量与道具购买非直接相关的隐性事件记录。
3. 玩家行为数据中游戏道具数量有限，一款互联网产品中物品数量成千上万，而一款网络游戏中的虚拟道具数量一般为几十或几百。

## 多示例多标记学习框架(Multi-Instance Multi-Label)

### 什么是MIML

真实世界的对象往往并不只具有唯一的语义，而是可能具有多义性的。如图像，网页等。
在MIML框架中,一个对象用多个示例描述,对象可以同时拥有多个类别标记。如下图
![](fig1.png)

### 为什么用MIML模型

1. 游戏提供商只能从游戏日志数据中直接观察到某位玩家在某段时间内经历的事件,以及他购买的游戏道具信息,却难以观察到事件与道具购买状态之间的直接关联信息,即难以获知是哪些事件导致了玩家对某道具的购买。
2. 传统的单示例机器学习算法进行推荐,必须给每一个示例(事件)指定一个标记(道具购买状态)作为训练样本。这样获得的训练样本具有极大噪音，因为该标记(道具购买状态)很可能并非由该示例(事件)“触发”。
3. 多示例多标记模型描述了一位经历了多个事件的玩家的所有道具购买状态。

我们认为游戏中的日志事件与道具购买状态之间的关联信息是构建道具推荐系统的基础。将每个事件作为一个示例，每个道具的购买状态作为一个类别标记，这样游戏道具推荐便对应到了我们的MIML框架。如下图：
![](fig2.png)

### 具体算法

![](fig3.png)

1. 为了在优化过程中利用不同标记之间的联系,先通过一个矩阵$W_0$ 将原来的特征向量$\mathbf{x}$映射到另一个空间里，这个空间对于各个标记是共享的,因而这个样本在标记$j$上的分类模型为:$$f_j(\mathbf{x})=\mathbf{w}_j^TW_0\mathbf{x}$$. 由于$W_0$和$\mathbf{w}_j$是通过交替优化进行的,不同标记之间的联系信息就可以通过$W_0$保存下来。
2. 为了应对MIML模型通常面临复杂语义的情形,算法为每个标记$j$设计了子概念类。令每个标记的子概念类数目为$K$则有$$f_j(\mathbf{x})=max_{k=1 ... K}f_{j,k}(\mathbf{x})＝ max_{k=1 ... K}\mathbf{w}_{j,k}W_0\mathbf{x}$$
注意到，max函数为模型引入了非线性
3. 在拥有了示例层面的模型后，需要建立以包为输入的模型，直接取包中示例的最大值$$f_j(X)=max_{\mathbf{x}\in X}f_j(\mathbf{x})$$
4. 有了对于包$X$在标记$j$上的模型，则定义这个模型的优化目标排序损失函数
![](fig4.png)
where $\alpha_i=1/i$.
显然,当与$X$相关的标记$j$被排得越靠后,所对应的排序损失值就越大,并且排序损失的权值是随着排序递减的,这样更能保证Top-n推荐时前n个标记的正确性。 
5. 采用随机梯度下降，每步优化的loss是
![](fig5.png)
更新梯度
![](fig6.png)
其中$err_{rank}$可以用采样方法近似$S_{i,v}$计算以加速

### 其他
可以使用co-training来利用未标记数据，co-training一开始应用于多视图的数据,后来被证明只要两个基分类器不同(掌握不同知识),就可以以未标记样本为载体来互相学习传递知识,并学习未标记的样本。将本输入数据利用Boostrap采样成两个数据集，训练出两个基分类器。